In [8]:
%run ./work/lib.py

In [9]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, DecimalType, TimestampType, DataType, IntegerType
from pyspark.sql.functions import col, lit, trim, substring, concat, udf, upper, initcap
from datetime import datetime
import os

In [10]:
conf = SparkConf() \
    .setAppName("projeto_pbi") \
    .setSparkHome('./spark/home')

LAKE_HOME = os.getenv("LAKE_HOME", "/spark/home")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

In [11]:
TABLE_NAME = 'fact_order_payment'

In [12]:
df = spark.read.csv('{}/dataset/e-commerce/01_extract/order_payments_dataset.csv'.format(LAKE_HOME), header=True)

df_payment = \
    df.select(
        col('order_id').cast(StringType()).alias('ID_ORDER'),
        col('payment_type').cast(StringType()).alias('DES_PAYMENT_TYPE'),
        col('payment_installments').cast(IntegerType()).alias('QTT_INSTALLMENT'),
        col('payment_value').cast(DecimalType(15,2)).alias('VAL_PAYMENT'),
    )

order_sk = map_sk(spark=spark, table_name='fact_order')

In [13]:
fact_order_payment = \
    df_payment \
        .withColumn('SK_ORDER', locate_sk(order_sk)(col('ID_ORDER')))

In [14]:
fact_order_payment = \
    fact_order_payment.select(
            col('SK_ORDER'),
            col('DES_PAYMENT_TYPE'),
            col('QTT_INSTALLMENT'),
            col('VAL_PAYMENT'),  
        )

fact_order_payment.write.mode('overwrite').csv('{}/dataset/e-commerce/04_fact/{}.csv'.format(LAKE_HOME, TABLE_NAME), header=True)

In [15]:
spark.stop()